# Downloading Audio with the RSS Feed

## Notebook Setup

In [1]:
# Importing the necessary Python libraries
import os
import json
import yaml

import feedparser
import mlx_whisper
import requests

ModuleNotFoundError: No module named 'feedparser'

In [ ]:
# Loading the RSS feed from file if file exists
if os.path.exists('../keys/watercooler_rss_feed.yaml'):
    with open('../keys/watercooler_rss_feed.yaml', 'r') as f:
        wc_rss_feed_url = yaml.safe_load(f)['WATERCOOLER_RSS_FEED']
else:
    wc_rss_feed_url = os.getenv('WATERCOOLER_RSS_FEED')

In [ ]:
# Instantiating the object representing the RSS feed
wc_rss_feed = feedparser.parse(wc_rss_feed_url)

In [ ]:
len(wc_rss_feed['entries'])

In [ ]:
# response = requests.get(wc_rss_feed['entries'][0]['links'][1]['href'], stream = True)

# with open('test.mp3', 'wb') as f:
#     for chunk in response.iter_content(chunk_size = 1024):
#         f.write(chunk)